<a href="https://colab.research.google.com/github/jasonstoy/TensorFlow/blob/main/14.%20NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-62f70688-ccb6-0484-1af3-0bb139d0efeb)


In [2]:
! wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-09-06 01:34:19--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-09-06 01:34:20 (76.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
! wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-09-06 01:34:22--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 74.125.23.128, 74.125.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.009s  

2021-09-06 01:34:22 (64.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

In [6]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_data_shuffled = train_df.sample(frac=1, random_state=5)
train_data_shuffled.head()

,id,keyword,location,text,target
6571,9404,survivors,"Chicago, IL",RT @kotowsa: South SudanÛªs war on women: sur...,1
4830,6877,mass%20murder,Everywhere,This Attempted Mass Murder brought to You by t...,0
2019,2898,damage,Your Conversation,This real shit will damage a bitch,0
7427,10625,wounded,NaN,Officer wounded suspect killed in exchange of ...,1
4983,7109,military,NaN,Hat #russian soviet army kgb military #cossac...,0


In [8]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
import random

random_index = random.randint(0,len(train_df)-5)
for row in train_data_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---")

Target: 0 (not real disaster)
Text:
Want to work at Swedish Health Services? We're #hiring in #Seattle WA! Click for details: http://t.co/4KDThCtEmV #Nursing #Job #Jobs

---
Target: 1 (real disaster)
Text:
@pmarca content is held hostage by network due to affiliation fees.

---
Target: 0 (not real disaster)
Text:
@LA_Siren Thanks for joining the foot. @VVorm

---
Target: 1 (real disaster)
Text:
Madhya Pradesh Train Derailment: Village Youth Saved Many Lives

---
Target: 0 (not real disaster)
Text:
@RohnertParkDPS You're on stage now! Right under the lights! Isn't it funny?! Where do you get the goofballs with which you staff your PD?

---


In [10]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_data_shuffled["text"].to_numpy(),
                                                                            train_data_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, 
                                                                            random_state=5)

In [11]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [12]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [13]:
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None)
                                    # pad_to_max_tokens=True)

In [14]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [15]:
#set up vectorization variables
max_vocab_length = 1000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=15)

In [16]:
#fit the vectorizer
text_vectorizer.adapt(train_sentences)

In [17]:
sample_sentence = "my test sentence"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=array([[13,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])>

In [18]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version:")

text_vectorizer([random_sentence])

Original text:
 being stuck on a sleeper train for 24 hours after de-railing due to a landslide was most definitely the pit of the trip        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[126,   1,  11,   3,   1, 121,  10,   1, 661,  41,   1, 500,   5,
          3, 332]])>

In [19]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
top_5_words

['', '[UNK]', 'the', 'a', 'in']

In [20]:
# embedding layer
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

In [21]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version:")

embedding(text_vectorizer([random_sentence]))

Original text:
 Breast milk is the original #superfood but rates worldwide have stalled below 40% contributing to more than 800000 child deaths last year.        

Vectorized version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0294896 , -0.02840542,  0.03157382, ...,  0.0075008 ,
         -0.00054563,  0.04900971],
        [-0.0294896 , -0.02840542,  0.03157382, ...,  0.0075008 ,
         -0.00054563,  0.04900971],
        [-0.01500814,  0.0027407 , -0.0401902 , ...,  0.0263404 ,
         -0.00567375, -0.00537703],
        ...,
        [ 0.03767825, -0.02605258,  0.0148719 , ...,  0.02769933,
         -0.01902651,  0.01417993],
        [-0.0294896 , -0.02840542,  0.03157382, ...,  0.0075008 ,
         -0.00054563,  0.04900971],
        [ 0.04000239, -0.03134932, -0.04276383, ..., -0.0154259 ,
          0.01827451, -0.04288304]]], dtype=float32)>

In [22]:
# baseline model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


In [23]:
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf",MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [24]:
baseline_score = model_0.score(val_sentences, val_labels)

In [25]:
baseline_score

0.7834645669291339

In [26]:
#make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1])

In [27]:
train_labels[:20]

array([1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0])

In [32]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calc_result(y_true, y_pred):
  #accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100

  #precision, recall, f1
  model_precision, model_recall, mode_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")

  model_results = {
      "accuracy":model_accuracy,
      "precision":model_precision,
      "recall":model_recall,
      "f1":mode_f1
  }

  return model_results

In [33]:
baseline_results = calc_result(y_true=val_labels,
                               y_pred=baseline_preds)

baseline_results

{'accuracy': 78.34645669291339,
 'f1': 0.7766167079582341,
 'precision': 0.7913267759468998,
 'recall': 0.7834645669291339}

In [37]:
# model 1 (simple dense model)
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

In [43]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [44]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           128000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 128,129
Trainable params: 128,129
Non-trainable params: 0
_________________________________________________________________


In [45]:
model_1.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_1 = model_1.fit(x=train_sentences,
                        y=train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210906-020425
Epoch 1/5
215/215 [==============================] - 2s 8ms/step - loss: 0.5911 - accuracy: 0.7135 - val_loss: 0.5309 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.4828 - accuracy: 0.7881 - val_loss: 0.4748 - val_accuracy: 0.7913
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.4373 - accuracy: 0.8070 - val_loss: 0.4585 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.4128 - accuracy: 0.8209 - val_loss: 0.4527 - val_accuracy: 0.7940
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3972 - accuracy: 0.8219 - val_loss: 0.4575 - val_accuracy: 0.7822


In [46]:
model_1_pred_probs = model_1.predict(val_sentences)

In [47]:
model_1_pred_probs.shape

(762, 1)

In [51]:
model_1_pred = tf.squeeze(tf.round(model_1_pred_probs))

In [52]:
model_1_result = calc_result(val_labels, model_1_pred)
model_1_result

{'accuracy': 78.21522309711287,
 'f1': 0.7779670274647487,
 'precision': 0.7841153800470415,
 'recall': 0.7821522309711286}

In [54]:
baseline_results

{'accuracy': 78.34645669291339,
 'f1': 0.7766167079582341,
 'precision': 0.7913267759468998,
 'recall': 0.7834645669291339}

In [56]:
import numpy as np

np.array(list(model_1_result.values())) > np.array(list(baseline_results.values()))

array([False, False, False,  True])

In [57]:
# visualizing embeddings
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(1000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [62]:
embed_weight = model_1.get_layer("embedding").get_weights()[0]
embed_weight.shape

(1000, 128)

In [64]:
# import io

# # Create output writers
# out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
# out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
# for num, word in enumerate(words_in_vocab):
#   if num == 0: 
#      continue # skip padding token
#   vec = embed_weight[num]
#   out_m.write(word + "\n") # write words to file
#   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
# out_v.close()
# out_m.close()

# # Download files locally to upload to Embedding Projector
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download("embedding_vectors.tsv")
#   files.download("embedding_metadata.tsv")

# http://projector.tensorflow.org/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [66]:
# LSTM Model

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [70]:
model_2.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [71]:
history_2 = model_2.fit(x=train_sentences,
                        y=train_labels,
                        validation_data = (val_sentences, val_labels),
                        epochs=5,
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210906-031414
Epoch 1/5
215/215 [==============================] - 10s 22ms/step - loss: 0.4088 - accuracy: 0.8221 - val_loss: 0.4818 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3794 - accuracy: 0.8340 - val_loss: 0.4810 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3739 - accuracy: 0.8364 - val_loss: 0.4835 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3557 - accuracy: 0.8470 - val_loss: 0.5251 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3311 - accuracy: 0.8568 - val_loss: 0.5181 - val_accuracy: 0.7638


In [74]:
model_2_pred = tf.squeeze(tf.round(model_2.predict(val_sentences)))

In [75]:
model_2_result = calc_result(val_labels, model_2_pred)
model_2_result

{'accuracy': 76.37795275590551,
 'f1': 0.7614611317036961,
 'precision': 0.7626691657124374,
 'recall': 0.7637795275590551}

In [77]:
# bidirectional LSTM

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_bidirectional_LSTM")

In [78]:
model_3.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [79]:
history_3 = model_3.fit(x=train_sentences,
                        y=train_labels,
                        validation_data = (val_sentences, val_labels),
                        epochs=5,
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_3_bidirectional_LSTM")])

Saving TensorBoard log files to: model_logs/model_3_bidirectional_LSTM/20210906-032115
Epoch 1/5
215/215 [==============================] - 14s 35ms/step - loss: 0.3585 - accuracy: 0.8498 - val_loss: 0.5284 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 5s 25ms/step - loss: 0.3124 - accuracy: 0.8632 - val_loss: 0.5678 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.2834 - accuracy: 0.8753 - val_loss: 0.5880 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 5s 24ms/step - loss: 0.2651 - accuracy: 0.8818 - val_loss: 0.6207 - val_accuracy: 0.7559
Epoch 5/5
215/215 [==============================] - 5s 25ms/step - loss: 0.2459 - accuracy: 0.8904 - val_loss: 0.6609 - val_accuracy: 0.7782


In [80]:
model_3_pred = tf.squeeze(tf.round(model_3.predict(val_sentences)))

In [81]:
model_3_result = calc_result(val_labels, model_3_pred)
model_3_result

{'accuracy': 77.82152230971128,
 'f1': 0.7732453276385931,
 'precision': 0.7812032148016453,
 'recall': 0.7782152230971129}

In [89]:
# Conv1D

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=10,
                  kernel_size=2,
                  strides=1,
                  padding="valid",
                  activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs)

In [90]:
model_4.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [91]:
history_4 = model_4.fit(x=train_sentences,
                        y=train_labels,
                        validation_data = (val_sentences, val_labels),
                        epochs=5,
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_4_Conv1D")])

Saving TensorBoard log files to: model_logs/model_4_Conv1D/20210906-033945
Epoch 1/5
215/215 [==============================] - 3s 10ms/step - loss: 0.4539 - accuracy: 0.8152 - val_loss: 0.4766 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3645 - accuracy: 0.8394 - val_loss: 0.5009 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3568 - accuracy: 0.8435 - val_loss: 0.5143 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3541 - accuracy: 0.8428 - val_loss: 0.5180 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3501 - accuracy: 0.8429 - val_loss: 0.5261 - val_accuracy: 0.7730


In [92]:
model_4_pred = tf.squeeze(tf.round(model_4.predict(val_sentences)))

In [93]:
model_4_result = calc_result(val_labels, model_4_pred)
model_4_result

{'accuracy': 77.29658792650919,
 'f1': 0.7718033837538939,
 'precision': 0.7718418538814236,
 'recall': 0.7729658792650919}

In [94]:
# use embedding from tensorflow hub
# Example of pretrained embedding with universal sentence encoder - https://tfhub.dev/google/universal-sentence-encoder/4
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [95]:
senstence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                         input_shape=[],
                                         dtype=tf.string,
                                         trainable=False,
                                         name="USE")

In [96]:
model_5 = tf.keras.Sequential([
  senstence_encoder_layer,
  layers.Dense(1,activation="sigmoid")
])

In [97]:
model_5.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [98]:
history_5 = model_5.fit(x=train_sentences,
                        y=train_labels,
                        validation_data = (val_sentences, val_labels),
                        epochs=5,
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_5_USE")])

Saving TensorBoard log files to: model_logs/model_5_USE/20210906-034820
Epoch 1/5
215/215 [==============================] - 9s 30ms/step - loss: 0.6437 - accuracy: 0.7535 - val_loss: 0.6019 - val_accuracy: 0.7966
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.5778 - accuracy: 0.7917 - val_loss: 0.5497 - val_accuracy: 0.7992
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.5364 - accuracy: 0.7943 - val_loss: 0.5155 - val_accuracy: 0.8045
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.5088 - accuracy: 0.7952 - val_loss: 0.4921 - val_accuracy: 0.8071
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4895 - accuracy: 0.7975 - val_loss: 0.4754 - val_accuracy: 0.8058


In [99]:
model_5_pred = tf.squeeze(tf.round(model_5.predict(val_sentences)))

In [100]:
model_5_result = calc_result(val_labels, model_5_pred)
model_5_result

{'accuracy': 80.5774278215223,
 'f1': 0.8051472368389099,
 'precision': 0.805075738258237,
 'recall': 0.8057742782152231}